In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.exceptions import ConvergenceWarning
import warnings

# Load the dataset
file_path = 'output_dataset.csv'  # Replace 'your_dataset.csv' with the actual file path
df = pd.read_csv(file_path)

# Drop non-numeric columns that are not needed for prediction
df = df.drop(['DayOfWeek', 'Date', 'UniqueCarrier', 'Airline', 'FlightNum', 'TailNum', 'Origin', 'Org_Airport', 'Dest', 'Dest_Airport', 'Cancelled', 'CancellationCode', 'Diverted'], axis=1)

# Define range-specific encoding function with 10 partitions
def range_encoding(value):
    if value <= 100:
        return '0-100'
    elif 101 <= value <= 200:
        return '101-200'
    elif 201 <= value <= 300:
        return '201-300'
    elif 301 <= value <= 400:
        return '301-400'
    elif 401 <= value <= 500:
        return '401-500'
    elif 501 <= value <= 600:
        return '501-600'
    elif 601 <= value <= 700:
        return '601-700'
    elif 701 <= value <= 800:
        return '701-800'
    elif 801 <= value <= 900:
        return '801-900'
    else:
        return '901-1000'

# Apply range-specific encoding to 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', and 'ArrDelay'
for col in ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'ArrDelay']:
    df[col] = df[col].apply(range_encoding)

# Convert categorical variables to numeric using Label Encoding
label_encoder = LabelEncoder()
for col in ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'ArrDelay']:
    df[col] = label_encoder.fit_transform(df[col])

# Separate features (X) and target variable (y)
X = df.drop('ArrDelay', axis=1)
y = df['ArrDelay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Suppress ConvergenceWarnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Create and train the ANN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5).astype(int).reshape(-1)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, zero_division=1)

# Display the results
print("\nANN Results:")
print("Accuracy:", accuracy)
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)

# Re-enable warnings after fitting the model
warnings.resetwarnings()


606/606 [==============================] - 1s 1ms/step

ANN Results:
Accuracy: 0.9496439995872459

Confusion Matrix:
[[15963   342     0     0     0     0     0     0     0]
 [   98  2443     0     0     0     0     0     0     0]
 [    0   419     0     0     0     0     0     0     0]
 [    0    89     0     0     0     0     0     0     0]
 [    0    19     0     0     0     0     0     0     0]
 [    0     6     0     0     0     0     0     0     0]
 [    0     1     0     0     0     0     0     0     0]
 [    0     1     0     0     0     0     0     0     0]
 [    0     1     0     0     0     0     0     0     0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     16305
           1       0.74      0.96      0.83      2541
           2       1.00      0.00      0.00       419
           3       1.00      0.00      0.00        89
           4       1.00      0.00      0.00        19
           5       1